In [1]:
import os
import numpy as np
import pandas as pd
import scipy
import librosa
import shutil
import keras


import keras.backend as K
import tensorflow as tf
from scipy import signal

from keras.optimizers import *

from keras.regularizers import *

from keras import regularizers, optimizers
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.utils import *
from keras.callbacks import *
from sklearn.model_selection import *
# from sklearn.cross_validation import StratifiedKFold
os.environ['CUDA_VISIBLE_DEVICES']=''
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF

config = tf.ConfigProto()  
config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
session = tf.Session(config=config)

# 设置session
KTF.set_session(session)
from keras import backend as K
K.get_session().list_devices()

os.listdir('../../audio-data/')
train_path = '../../audio-data/audio_train/'
test_path = '../../audio-data/audio_test/'
train = pd.read_csv('../../audio-data/train.csv')
test = pd.read_csv('../../audio-data/sample_submission.csv')
# print('training samples: ', len(os.listdir(train_path)))
# print('test samples: ', len(os.listdir(test_path)))
# print('training labels: ', len(train.label.unique()))
# print(train.head())
LABELS = list(train.label.unique())
label_idx = {label: i for i, label in enumerate(LABELS)}
train.set_index('fname', inplace=True)
test.set_index('fname', inplace=True)
train['label_idx'] = train.label.apply(lambda x: label_idx[x])






/home/li/anaconda3/envs/audio/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/li/anaconda3/envs/audio/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


[_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456)]

['train.csv',
 'audio_test.zip',
 'audio_train',
 'sample_submission.csv',
 'audio_test',
 'audio_train.zip',
 'Samples']

In [2]:
def get_acc_loss(path, n_folds):
    acc = loss = 0
    for i in range(n_folds):
        if n_folds == 9 and i == 1:
            continue
            
        log = pd.read_csv(path + '/log_%d.csv'%i)

        loss += np.min(log.val_loss)
        idx = np.argmin(log.val_loss)
        acc += log.val_acc[idx]
#         print(np.min(log.val_loss), log.val_acc[idx])
    return acc/n_folds, loss/n_folds
path = []
path.append(['predictions_log_mel_sp_44000x2/', 7])
path.append(['predictions_log_mel_sp_44000x2_mixup/', 7])
path.append(['predictions_log_mel_sp_44000x2_image_aug/', 7])

path.append(['predictions_log_mel_sp_24000x5/', 9])
path.append(['predictions_log_mel_sp_24000x5_mixup/', 7])
path.append(['predictions_log_mel_sp_24000x5_image_gen/', 7])
path.append(['predictions_24000x2_log_sp/', 7])
path.append(['predictions_log_sp_24000x2_mixup/', 7])
path.append(['predictions_log_sp_24000x2_image_gen/', 5])
path.append(['predictions_cqt_24000x3/', 5])
path.append(['predictions_cqt_24000x3_mixup/', 5])
path.append(['predictions_cqt_24000x3_img_aug/', 5])
path.append(['predictions_mfcc_44100x5_img_aug', 5])
path.append(['predictions_log_mel_sp_44100x6_img_aug/', 5])
path.append(['predictions_vggish/', 5])
path.append(['predictions_log_mel_sp_44000x5_image_aug_10f/', 10])
path.append(['predictions_log_mel_sp_44100x5_kfold-10/', 10])
path.append(['predictions_log_mel_sp_44100x5_kfold/', 5])
path.append(['predictions_log_sp_44100x5_img_aug/', 10])
for i in range(len(path)):
    acc , loss = get_acc_loss(path[i][0] , path[i][1])
    print(path[i], 'acc :', acc, 'loss: ', loss)
#     print('loss: ', loss)
!pwd

/home/li/anaconda3/envs/audio/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)


['predictions_log_mel_sp_44000x2/', 7] acc : 0.7797247247197147 loss:  0.9303560311770204
['predictions_log_mel_sp_44000x2_mixup/', 7] acc : 0.7893353493933226 loss:  0.844312639151507
['predictions_log_mel_sp_44000x2_image_aug/', 7] acc : 0.7900653655000961 loss:  0.8313511508192256
['predictions_log_mel_sp_24000x5/', 9] acc : 0.7060456569477587 loss:  0.7577261961597551
['predictions_log_mel_sp_24000x5_mixup/', 7] acc : 0.8605291843245098 loss:  0.5686654499304071
['predictions_log_mel_sp_24000x5_image_gen/', 7] acc : 0.7869222531054573 loss:  0.8446435530662757
['predictions_24000x2_log_sp/', 7] acc : 0.7452706153910559 loss:  0.9893222789126284
['predictions_log_sp_24000x2_mixup/', 7] acc : 0.7924579965709134 loss:  0.8170173985621155
['predictions_log_sp_24000x2_image_gen/', 5] acc : 0.7191866106542538 loss:  1.088727391592129
['predictions_cqt_24000x3/', 5] acc : 0.4810647575568975 loss:  2.0045152819596983
['predictions_cqt_24000x3_mixup/', 5] acc : 0.5030904197667818 loss:  1.8

In [3]:
def get_acc_loss(path, n_folds):
    acc = loss = 0
    for i in range(n_folds):
        if n_folds == 9 and i == 1:
            continue
            
        log = pd.read_csv(path + '/log_%d.csv'%i)

        loss += np.min(log.val_loss)
        idx = np.argmin(log.val_loss)
        acc += log.val_acc[idx]
#         print(np.min(log.val_loss), log.val_acc[idx])
    return acc/n_folds, loss/n_folds
path = []
for i in range(5):
    path.append(['predictions_log_mel_sp_44100x5_5x2fold-%d/'%i, 2])
for i in range(len(path)):
    acc , loss = get_acc_loss(path[i][0] , path[i][1])
    print(path[i], 'acc :', acc, 'loss: ', loss)

['predictions_log_mel_sp_44100x5_5x2fold-0/', 2] acc : 0.7598431509208703 loss:  0.9278248513366016
['predictions_log_mel_sp_44100x5_5x2fold-1/', 2] acc : 0.7664934418665024 loss:  0.922063210272247
['predictions_log_mel_sp_44100x5_5x2fold-2/', 2] acc : 0.7627993657249714 loss:  0.9201675675520022
['predictions_log_mel_sp_44100x5_5x2fold-3/', 2] acc : 0.7569940259686538 loss:  0.9362373268672838
['predictions_log_mel_sp_44100x5_5x2fold-4/', 2] acc : 0.7676561412386104 loss:  0.9004897201738815


/home/li/anaconda3/envs/audio/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)


In [57]:

pred_list_mixup = []
pred_list = []

for i in range(10):
    pred_list_mixup.append(np.load("predictions_log_sp_44100x5_img_aug///test_predictions_%d.npy"%i))
for i in range(10):
    pred_list.append(np.load('predictions_log_mel_sp_44000x5_image_aug_10f/test_predictions_%d.npy'%i))
# for i in range(5):
#     for j in range(2):
#         pred_list.append(np.load('predictions_log_mel_sp_44100x5_5x2fold-%d/test_predictions_%d.npy'%(i, j)))
# for i in range(10):
#     pred_list.append(np.load('../predictions_vgg16_melspec/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list.append(np.load("../predictions_1d_24000x5/test_predictions_%d.npy"%i))
# for i in range(7):
#     pred_list.append(np.load('predictions_24000x2_log_sp/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list.append(np.load('predictions_log_mel_sp_44000x5/test_predictions_%d.npy'%i))

# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_sp_24000x2_mixup/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_sp_24000x2_image_gen/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_mel_sp_24000x5_image_gen/test_predictions_%d.npy'%i))
# for i in range(10):
#     pred_list_mixup.append(np.load('predictions_log_mel_sp_24000x5_mixup/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_mel_sp_44000x2_image_aug/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_mel_sp_44000x5_mixup/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_mel_sp_44000x5_image_aug/test_predictions_%d.npy'%i))
    
prediction_mixup = np.ones_like(pred_list_mixup[0])
prediction = np.ones_like(pred_list[0])



for pred in pred_list_mixup:
    prediction_mixup = prediction_mixup * pred
for pred in pred_list:
    prediction = prediction * pred

    
prediction_mixup = prediction_mixup**(1./len(pred_list_mixup))
prediction = prediction**(1./len(pred_list))


prediction = prediction + prediction_mixup

# Make a submission file
top_3 = np.array(LABELS)[np.argsort(-prediction, axis=1)[:, :3]]
predicted_labels = [' '.join(list(x)) for x in top_3]
test = pd.read_csv('../../audio-data/sample_submission.csv')
test['label'] = predicted_labels
test[['fname', 'label']].to_csv("lms-ls.csv", index=False)



!du -ah lms-ls.csv
!pwd



380K	lms-ls.csv
/disk1/audio-tagging/notebook


In [57]:

pred_list_mixup = []
pred_list = []
p1_list = p2_list = p3_list = p4_list = p5_list = []

# for i in range(10):
#     p1_list.append(np.load('../predictions_vgg16_melspec/test_predictions_%d.npy'%i))
for i in range(7):
    p2_list.append(np.load("../predictions_1d_24000x5/test_predictions_%d.npy"%i))
# for i in range(7):
#     p3_list.append(np.load('predictions_24000x2_log_sp/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list.append(np.load('predictions_log_mel_sp_24000x5/test_predictions_%d.npy'%i))

# for i in range(7):
#     p4_list_mixup.append(np.load('predictions_log_sp_24000x2_mixup/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_mel_sp_24000x5_image_gen/test_predictions_%d.npy'%i))
for i in range(10):
    p4_list.append(np.load('predictions_log_mel_sp_24000x5_mixup/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_mel_sp_44000x5_mixup/test_predictions_%d.npy'%i))
for i in range(7):
    p5_list.append(np.load('predictions_log_mel_sp_44000x5_image_aug/test_predictions_%d.npy'%i))
    
# prediction_mixup = np.ones_like(pred_list_mixup[0])
# prediction = np.ones_like(pred_list[0])
p1 = np.ones_like(p1_list[0])
p2 = np.ones_like(p2_list[0])
p3 = np.ones_like(p3_list[0])
p4 = np.ones_like(p4_list[0])
p5 = np.ones_like(p5_list[0])



# for pred in pred_list_mixup:
#     prediction_mixup = prediction_mixup * pred
# for pred in pred_list:
#     prediction = prediction * pred
for p in p1_list:
    p1 *= p
for p in p2_list:
    p2 *= p
for p in p3_list:
    p3 *= p
for p in p4_list:
    p4 *= p
for p in p5_list:
    p5 *= p

    
# prediction_mixup = prediction_mixup**(1./len(pred_list_mixup))
# prediction = prediction**(1./len(pred_list))
p1 = p1 ** (1./len(p1_list))
p2 = p2 ** (1./len(p2_list))
p3 = p3 ** (1./len(p3_list))
p4 = p4 ** (1./len(p4_list))
p5 = p5 ** (1./len(p5_list))

# prediction = prediction + prediction_mixup
# p1 > p5 > p4 > p3 > p2
prediction = 0.5 * (p1 + p2 + p3) + p4 + p5

# Make a submission file
top_3 = np.array(LABELS)[np.argsort(-prediction, axis=1)[:, :3]]
predicted_labels = [' '.join(list(x)) for x in top_3]
test = pd.read_csv('../../audio-data/sample_submission.csv')
test['label'] = predicted_labels
test[['fname', 'label']].to_csv("p1-p5.csv", index=False)





!pwd

/media/li/badc7046-a528-4bae-be3f-319caa281543/audio-tagging/notebook


In [11]:

pred_list_mixup = []
pred_list = []

# for i in range(10):
#     pred_list.append(np.load("predictions_log_mel_sp_24000x5_mixup/test_predictions_%d.npy"%i))

for i in range(10):
    pred_list.append(np.load('../predictions_vgg16_melspec/test_predictions_%d.npy'%i))
for i in range(7):
    pred_list.append(np.load("../predictions_1d_24000x5/test_predictions_%d.npy"%i))
# for i in range(7):
#     pred_list.append(np.load('predictions_24000x2_log_sp/test_predictions_%d.npy'%i))
    
# for i in range(10):
#     pred_list.append(np.load('predictions_log_mel_sp_44100x5_kfold-10/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list.append(np.load('predictions_log_mel_sp_44000x5/test_predictions_%d.npy'%i))
for i in range(7):
    pred_list.append(np.load('predictions_log_mel_sp_44000x2/test_predictions_%d.npy'%i))
# for i in range(10):
#     pred_list_mixup.append(np.load('predictions_log_sp_44100x5_img_aug/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_sp_24000x2_mixup/test_predictions_%d.npy'%i))
for i in range(7):
    pred_list_mixup.append(np.load('predictions_log_mel_sp_24000x5_image_gen/test_predictions_%d.npy'%i))
for i in range(7):
    pred_list_mixup.append(np.load('predictions_log_mel_sp_24000x5_mixup/test_predictions_%d.npy'%i))
for i in range(7):
    pred_list_mixup.append(np.load('predictions_log_mel_sp_44100x6_img_aug///test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_mel_sp_44000x5_image_aug_10f/test_predictions_%d.npy'%i))
# for i in range(5):
#     pred_list_mixup.append(np.load('predictions_log_mel_sp_44100x5_kfold-10///test_predictions_%d.npy'%i))
for i in range(7):
    pred_list_mixup.append(np.load('predictions_log_mel_sp_44000x2_image_aug/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_mel_sp_44000x5_mixup/test_predictions_%d.npy'%i))
# for i in range(7):
#     pred_list_mixup.append(np.load('predictions_log_mel_sp_44000x5_image_aug/test_predictions_%d.npy'%i))

# for i in range(5):
#     pred_list_mixup.append(np.load('predictions_mfcc_44100x5_img_aug/test_predictions_%d.npy'%i))
    
prediction_mixup = np.ones_like(pred_list_mixup[0])
prediction = np.ones_like(pred_list[0])



for pred in pred_list_mixup:
    prediction_mixup = prediction_mixup * pred
for pred in pred_list:
    prediction = prediction * pred

    
prediction_mixup = prediction_mixup**(1./len(pred_list_mixup))
prediction = prediction**(1./len(pred_list))


prediction = prediction + prediction_mixup

# Make a submission file
top_3 = np.array(LABELS)[np.argsort(-prediction, axis=1)[:, :3]]
predicted_labels = [' '.join(list(x)) for x in top_3]
test = pd.read_csv('../../audio-data/sample_submission.csv')
test['label'] = predicted_labels
test[['fname', 'label']].to_csv("sad.csv", index=False)



!du -ah sad.csv




344K	sad.csv
